In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import numpy as np
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("final2.csv")  # Load your dataset

In [ ]:
df.columns = df_clean.columns.str.replace(' ', '_')  # Replace spaces with underscores in column names

In [ ]:
df.head(5)

,Alloy,Finish,Length,Weight,Tolerances,GD&T,Order_Quantity,LME_price,Customer_Category,Lead_Time_(weeks),Quote_Price_(SEK),Quote_Date,Profile_Name,Profile_Complexity,Manufacturing_Difficulty,LME_MA_7,LME_Lag_1
0,0.0,4.0,17.6,1.718,0.147,0.0,50862,2.75,2.0,11.0,3.83,2025-01-01,3.014848,3,0.147,2.750000,2.75
1,1.0,3.0,27.0,1.245,0.067,2.0,116471,2.96,1.0,9.0,2.67,2025-01-01,3.026308,3,0.201,2.855000,2.75
2,2.0,4.0,24.1,0.885,0.099,2.0,63929,3.97,2.0,7.0,3.60,2025-01-02,2.951505,1,0.297,3.226667,2.96
3,2.0,4.0,26.1,1.042,0.152,1.0,120204,2.83,3.0,10.0,2.61,2025-01-02,3.160254,5,0.304,3.127500,3.97
4,2.0,3.0,34.8,1.368,0.118,0.0,94419,3.59,1.0,2.0,3.06,2025-01-03,2.987982,1,0.118,3.220000,2.83


In [ ]:
# Convert Quote Date to datetime and sort
df["Quote_Date"] = pd.to_datetime(df["Quote_Date"])
df = df.sort_values("Quote_Date").reset_index(drop=True)

# Separate features and target
y = df["Quote_Price_(SEK)"]
X = df.drop(columns=["Quote_Price_(SEK)", "Quote_Date"])

# TimeSeries cross-validation
tscv = TimeSeriesSplit(n_splits=5)
results = []
fold = 1

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    results.append({
        "Fold": fold,
        "MAE": round(mean_absolute_error(y_test, preds), 4),
        "RMSE": round(np.sqrt(mean_squared_error(y_test, preds)), 4),
        "R2 Score": round(r2_score(y_test, preds), 4),
        "MAPE (%)": round(mean_absolute_percentage_error(y_test, preds) * 100, 2)
    })
    fold += 1

results_df = pd.DataFrame(results)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000171 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1697
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 15
[LightGBM] [Info] Start training from score 3.016937
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [ ]:
results_df